Gerador de Dashboards automatico

O script deve:

analisar a base de dados;
extrair quais informações são mais relevantes;
sugerir formas de visualizar essas informações;
criar as visualizações usando plotly.express
decidir a organização delas;


1º passo importar a base de dados que sera usada e transforma-la em uma string para que o gemini interprete ela

In [145]:
import pandas as pd

df = pd.read_csv('waterquality.csv')
display(df)

df_str = df.to_string()
display(df_str)

,Date,Salinity (ppt),DissolvedOxygen (mg/L),pH,SecchiDepth (m),WaterDepth (m),WaterTemp (C),AirTemp (C)
0,1989-05-11,NaN,NaN,7.5,0.3,0.9,17.0,-17.777778
1,1989-05-18,NaN,12.0,7.5,0.2,0.6,17.5,20.500000
2,1989-05-25,NaN,NaN,8.0,0.4,0.8,23.0,25.000000
3,1989-06-01,NaN,12.0,8.0,0.4,0.9,25.5,29.000000
4,1989-07-11,NaN,NaN,8.5,0.3,0.9,28.5,28.000000
...,...,...,...,...,...,...,...,...
2366,NaN,0.0,NaN,7.0,0.3,0.5,14.0,11.111111
2367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-17.777778
2368,NaN,0.0,3.9,6.5,0.3,1.8,22.0,21.111111
2369,NaN,1.0,NaN,7.0,0.6,1.0,17.0,21.111111


'            Date  Salinity (ppt)  DissolvedOxygen (mg/L)    pH  SecchiDepth (m)  WaterDepth (m)  WaterTemp (C)  AirTemp (C)\n0     1989-05-11             NaN                     NaN  7.50            0.300           0.900          17.00   -17.777778\n1     1989-05-18             NaN                   12.00  7.50            0.200           0.600          17.50    20.500000\n2     1989-05-25             NaN                     NaN  8.00            0.400           0.800          23.00    25.000000\n3     1989-06-01             NaN                   12.00  8.00            0.400           0.900          25.50    29.000000\n4     1989-07-11             NaN                     NaN  8.50            0.300           0.900          28.50    28.000000\n5     1989-07-20             NaN                    1.60  9.00            0.300           0.900          30.00    32.500000\n6     1989-07-27             NaN                    6.20  9.00            0.300           0.800          27.00    27.000000\

2º passo importar a biblioteca do gemini e configurar o modelo

In [146]:
import google.generativeai as genai

# Configuração da chave da API do Google Generative AI
genai.configure(api_key="Sua_key")

# Configuração de geração de texto
generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 18192,
    "candidate_count": 1
}
# Configuração de segurança para evitar conteúdo ofensivo
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
]
# Criação do modelo de geração de texto
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    safety_settings=safety_settings
)



3º passo começar um chat pedir para ele interpretar o dataframe e exeplicar para ele informações relevantes

In [159]:
convo = model.start_chat(history=[])
prompt_parts=[
        "A seguir eu te passarei um dataframe e quero que você me diga as informações que cada coluna possui:\n",
        "Você deve seguir a risca as informações presentes nesse dataframe, e não deve alterar nenhum nome, principalmente o nome das colunas:\n",
        df_str, 
        "\n",
    ]
convo.send_message(prompt_parts)
print(convo.last.text)
convo.send_message('Baseado nessas informações me de sugestões de graficos que eu posso gerar, e quais colunas usar nesse grafico\n')
convo.send_message("Esses são os tipos de graficos suportados, você só pode usar eles: 'linha','barra','scatter','histograma','box','violin','area','pie','density_mapbox','choropleth_mapbox','scatter_mapbox','line_mapbox','bar_mapbox'")


O dataframe que você forneceu contém as seguintes colunas:

* **Date:** Data da coleta da amostra de água, no formato AAAA-MM-DD.
* **Salinity (ppt):** Salinidade da água, medida em partes por mil (ppt).
* **DissolvedOxygen (mg/L):** Concentração de oxigênio dissolvido na água, medida em miligramas por litro (mg/L).
* **pH:** pH da água, que indica a acidez ou alcalinidade da água.
* **SecchiDepth (m):** Profundidade de Secchi, medida em metros (m), que indica a transparência da água.
* **WaterDepth (m):** Profundidade da água no local da coleta da amostra, medida em metros (m).
* **WaterTemp (C):** Temperatura da água, medida em graus Celsius (C).
* **AirTemp (C):** Temperatura do ar no momento da coleta da amostra, medida em graus Celsius (C). 



response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "Certo! Considerando os tipos de gráficos que você listou, aqui estão algumas sugestões de gráficos que você pode gerar e as colunas a serem usadas:\n\n**Análise Temporal:**\n\n1. **'linha'**: \n    * **Colunas:** `Date` (eixo x) e uma das seguintes (eixo y): `Salinity (ppt)`, `DissolvedOxygen (mg/L)`, `pH`, `SecchiDepth (m)`, `WaterDepth (m)`, `WaterTemp (C)`, `AirTemp (C)`.\n    * **Descrição:** Visualize a tendência temporal de cada parâmetro individualmente. Por exemplo, a variação da temperatura da água ao longo dos anos.\n\n2. **'area'**:\n    * **Colunas:** `Date` (eixo x) e uma ou mais das seguintes (eixo y): `Salinity (ppt)`, `DissolvedOxygen (mg/L)`, `pH`, `SecchiDepth (m)`, `WaterDepth (m)`, `WaterTemp (C)`, `AirTemp (C)`.\n    * **Descrição:**  Similar ao gráfico de linha, mas a área sob a curva é preenchida, o que pode 

4º pedir para ele me dar 5 exemplos de graficos baseado no df no formato de um dicionario com as informações que usarei para criar os graficos

In [160]:
convo.send_message("Você deve me trazer 5 respostas em forma um dicionario como no exemplo: {'tipo_de_grafico':'o tipo', 'coluna_x':'o_nome_da_coluna_no_df','coluna_y':'o_nome_da_coluna_no_df'}. A sua resposta deve conter unicamente o dicionario, sem qualquer outro tipo de informação")
dicionario=convo.last.text
print(dicionario)

```python
[
{'tipo_de_grafico': 'linha', 'coluna_x': 'Date', 'coluna_y': 'WaterTemp (C)'},
{'tipo_de_grafico': 'scatter', 'coluna_x': 'Salinity (ppt)', 'coluna_y': 'DissolvedOxygen (mg/L)'},
{'tipo_de_grafico': 'histograma', 'coluna_x': 'pH', 'coluna_y': None},
{'tipo_de_grafico': 'box', 'coluna_x': 'Date', 'coluna_y': 'SecchiDepth (m)'},
{'tipo_de_grafico': 'area', 'coluna_x': 'Date', 'coluna_y': 'DissolvedOxygen (mg/L)'}
]
```


5º passo pedir para ele gerar uma lista com uma breve descrição do que aquele grafico representa 

In [161]:
convo.send_message("crie uma lista que possui um texto descritivo das informações que estão presentes em cada grafico com uma breve analise sobre ele")
lista=convo.last.text
print(lista)

```python
[
"Gráfico de linha mostrando a variação da temperatura da água ('WaterTemp (C)') ao longo do tempo ('Date'). Permite analisar as flutuações de temperatura, identificar padrões sazonais e tendências de aquecimento ou resfriamento.",
"Gráfico de dispersão correlacionando a salinidade ('Salinity (ppt)') com a concentração de oxigênio dissolvido ('DissolvedOxygen (mg/L)').  Ajuda a entender a relação entre esses dois parâmetros, verificando se há correlação positiva, negativa ou ausência de correlação.",
"Histograma da distribuição de frequência do pH da água ('pH'). Revela a frequência de cada valor de pH, indicando se a água é predominantemente ácida, neutra ou alcalina.",
"Boxplot comparando a distribuição da profundidade de Secchi ('SecchiDepth (m)') em diferentes períodos de tempo ('Date').  Permite visualizar a variabilidade da transparência da água ao longo do tempo, identificando períodos com maior ou menor turbidez.",
"Gráfico de área ilustrando a variação da concentraç

6º passo separar o dicionario e a lista para transforma-la de string para os seus respectivos tipos.                                                                Criar um dicionario para transformar o tipo de gráfico no padrão aceito peolo px.                                                                                  Criar e plotar cada um dos graficos sugeridos com a sua respectiva descrição.

In [162]:
import ast
import re 
matches = re.findall(r'{.*}', dicionario)

lista = lista.replace("\n", "") 
pattern = r'\[([^\]]+)\]'  
matches2 = [match.group(1) for match in re.finditer(pattern, lista)]
matches2=matches2[0].replace('",', "/") 
matches2=matches2.replace('"', "") 
matches2=matches2.split('/')


graph_types = {
    'linha': px.line,
    'barra': px.bar,
    'scatter': px.scatter,
    'histograma': px.histogram,
    'box': px.box,
    'violin': px.violin,
    'area': px.area,
    'pie': px.pie,
    'density_mapbox': px.density_mapbox,
    'choropleth_mapbox': px.choropleth_mapbox,
    'scatter_mapbox': px.scatter_mapbox,
    'line_mapbox': px.line_mapbox
}

for i in range(len(matches)):
    
    dict_from_string = ast.literal_eval(matches[i])
    display(dict_from_string)
    graph_func = graph_types.get(dict_from_string['tipo_de_grafico'])
    if graph_func:
        fig = graph_func(df, x=dict_from_string['coluna_x'], y=dict_from_string['coluna_y'],color_discrete_sequence=['#2CC9D5'])
        
        fig.update_layout(
        paper_bgcolor="#242424",
        plot_bgcolor="#242424",
        autosize=True,
        title_font=dict(color='white'),
        xaxis=dict(color='white'),
        yaxis=dict(color='white'),
        legend=dict(bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)', font=dict(color='white')),
        margin=dict(l=10, r=10, b=10, t=40)
        )
        fig.show()
        display(matches2[i])
    else:
        print(f"Unsupported graph type: {dict_from_string['tipo_de_grafico']}")

{'tipo_de_grafico': 'linha', 'coluna_x': 'Date', 'coluna_y': 'WaterTemp (C)'}

"Gráfico de linha mostrando a variação da temperatura da água ('WaterTemp (C)') ao longo do tempo ('Date'). Permite analisar as flutuações de temperatura, identificar padrões sazonais e tendências de aquecimento ou resfriamento."

{'tipo_de_grafico': 'scatter',
 'coluna_x': 'Salinity (ppt)',
 'coluna_y': 'DissolvedOxygen (mg/L)'}

"Gráfico de dispersão correlacionando a salinidade ('Salinity (ppt)') com a concentração de oxigênio dissolvido ('DissolvedOxygen (mg"

{'tipo_de_grafico': 'histograma', 'coluna_x': 'pH', 'coluna_y': None}

"L)').  Ajuda a entender a relação entre esses dois parâmetros, verificando se há correlação positiva, negativa ou ausência de correlação."

{'tipo_de_grafico': 'box', 'coluna_x': 'Date', 'coluna_y': 'SecchiDepth (m)'}

"Histograma da distribuição de frequência do pH da água ('pH'). Revela a frequência de cada valor de pH, indicando se a água é predominantemente ácida, neutra ou alcalina."

{'tipo_de_grafico': 'area',
 'coluna_x': 'Date',
 'coluna_y': 'DissolvedOxygen (mg/L)'}

"Boxplot comparando a distribuição da profundidade de Secchi ('SecchiDepth (m)') em diferentes períodos de tempo ('Date').  Permite visualizar a variabilidade da transparência da água ao longo do tempo, identificando períodos com maior ou menor turbidez."

7º passo, caso sejá necessario criar mais 5 gráficos seguindo o mesmo fluxo. Essa etapa pode ser repetida quantas vezes o usuario quiser até encontrar graficos que lhe interesse

In [163]:
convo.send_message("Me traga mais 5 dicionarios, continue usando unicamente as colunas do df analisado anteriormente")
dicionario=convo.last.text
print(dicionario)

```python
[
{'tipo_de_grafico': 'violin', 'coluna_x': 'Date', 'coluna_y': 'WaterTemp (C)'},
{'tipo_de_grafico': 'barra', 'coluna_x': 'Date', 'coluna_y': 'Salinity (ppt)'},
{'tipo_de_grafico': 'linha', 'coluna_x': 'Date', 'coluna_y': 'AirTemp (C)'},
{'tipo_de_grafico': 'scatter', 'coluna_x': 'WaterDepth (m)', 'coluna_y': 'WaterTemp (C)'},
{'tipo_de_grafico': 'histograma', 'coluna_x': 'SecchiDepth (m)', 'coluna_y': None}
]
```


In [164]:
convo.send_message("Crie mais uma lista de descrição para o segundo conjunto")
lista=convo.last.text
print(lista)

```python
[
"Gráfico de violino exibindo a distribuição da temperatura da água ('WaterTemp (C)') em diferentes períodos de tempo ('Date').  Permite comparar a forma e a dispersão da distribuição da temperatura em diferentes anos, meses ou estações do ano.",
"Gráfico de barras mostrando a variação da salinidade ('Salinity (ppt)') ao longo do tempo ('Date').  Útil para visualizar as mudanças na salinidade em diferentes períodos, identificando aumentos ou diminuições significativas.",
"Gráfico de linha representando a variação da temperatura do ar ('AirTemp (C)') ao longo do tempo ('Date'). Permite analisar as flutuações da temperatura do ar e identificar padrões sazonais.",
"Gráfico de dispersão correlacionando a profundidade da água ('WaterDepth (m)') com a temperatura da água ('WaterTemp (C)'). Ajuda a investigar se há uma relação entre a profundidade e a temperatura da água, como uma possível estratificação térmica.",
"Histograma da distribuição de frequência da profundidade de Secchi

In [165]:
matches = re.findall(r'{.*}', dicionario)

lista = lista.replace("\n", "") 
pattern = r'\[([^\]]+)\]'  
matches2 = [match.group(1) for match in re.finditer(pattern, lista)]
matches2=matches2[0].replace('",', "/") 
matches2=matches2.replace('"', "") 
matches2=matches2.split('/')

for i in range(len(matches)):
    
    dict_from_string = ast.literal_eval(matches[i])
    display(dict_from_string)
    graph_func = graph_types.get(dict_from_string['tipo_de_grafico'])
    if graph_func:
        fig = graph_func(df, x=dict_from_string['coluna_x'], y=dict_from_string['coluna_y'])
        fig.update_layout(
        paper_bgcolor="#242424",
        plot_bgcolor="#242424",
        autosize=True,
        title_font=dict(color='white'),
        xaxis=dict(color='white'),
        yaxis=dict(color='white'),
        legend=dict(bgcolor='rgba(0,0,0,0)', bordercolor='rgba(0,0,0,0)', font=dict(color='white')),
        margin=dict(l=10, r=10, b=10, t=40)
        )
        fig.show()
        display(matches2[i])
    else:
        print(f"Unsupported graph type: {dict_from_string['tipo_de_grafico']}")

{'tipo_de_grafico': 'violin', 'coluna_x': 'Date', 'coluna_y': 'WaterTemp (C)'}

"Gráfico de violino exibindo a distribuição da temperatura da água ('WaterTemp (C)') em diferentes períodos de tempo ('Date').  Permite comparar a forma e a dispersão da distribuição da temperatura em diferentes anos, meses ou estações do ano."

{'tipo_de_grafico': 'barra', 'coluna_x': 'Date', 'coluna_y': 'Salinity (ppt)'}

"Gráfico de barras mostrando a variação da salinidade ('Salinity (ppt)') ao longo do tempo ('Date').  Útil para visualizar as mudanças na salinidade em diferentes períodos, identificando aumentos ou diminuições significativas."

{'tipo_de_grafico': 'linha', 'coluna_x': 'Date', 'coluna_y': 'AirTemp (C)'}

"Gráfico de linha representando a variação da temperatura do ar ('AirTemp (C)') ao longo do tempo ('Date'). Permite analisar as flutuações da temperatura do ar e identificar padrões sazonais."

{'tipo_de_grafico': 'scatter',
 'coluna_x': 'WaterDepth (m)',
 'coluna_y': 'WaterTemp (C)'}

"Gráfico de dispersão correlacionando a profundidade da água ('WaterDepth (m)') com a temperatura da água ('WaterTemp (C)'). Ajuda a investigar se há uma relação entre a profundidade e a temperatura da água, como uma possível estratificação térmica."

{'tipo_de_grafico': 'histograma',
 'coluna_x': 'SecchiDepth (m)',
 'coluna_y': None}

"Histograma da distribuição de frequência da profundidade de Secchi ('SecchiDepth (m)').  Indica a frequência de cada valor de profundidade de Secchi, revelando a predominância de águas mais ou menos transparentes."